## Goals and Questions

This notebook is designed to help me build Magic: The Gathering decks using external data from the website "Scryfall," which tracks datapoints on all cards, such as set, rarity, price, and more.

In building this, I hope to:

* Better understand my current decks
* Budget more effectively for little squares of cardboard in upcoming decks through automation of price fetchings API to get me prices. 
* Visualize all this information in a Power BI dashboard. 

### I have a number of questions:

* What are some "key words" for each deck?
* What does the "mana curve" (the cost to cast each card, compared to other cards) look like?
* Which deck is most expensive? Does this have any relationship to how many different "colors" are in that deck (spoilers, the answer is almost certainly "Yes")
* What trends and patterns am I *not* seeing yet?

I am also evaluating using a task scheduler to execute this process on a set schedule. This would be a slightly different project, meant to track card prices for cards I *don't* own yet, informing decisions about when to purchase them.

In [2]:
import requests as requests
import pandas as pd

In [3]:
# An initial exploration into Scryfall's "bulk" repository.

url = 'https://api.scryfall.com/bulk-data'
headers = {'Content-type': 'application/json'}
r = requests.get(url, headers=headers)
r.text

'{"object":"list","has_more":false,"data":[{"object":"bulk_data","id":"27bf3214-1271-490b-bdfe-c0be6c23d02e","type":"oracle_cards","updated_at":"2023-04-14T09:02:08.589+00:00","uri":"https://api.scryfall.com/bulk-data/27bf3214-1271-490b-bdfe-c0be6c23d02e","name":"Oracle Cards","description":"A JSON file containing one Scryfall card object for each Oracle ID on Scryfall. The chosen sets for the cards are an attempt to return the most up-to-date recognizable version of the card.","size":114511789,"download_uri":"https://data.scryfall.io/oracle-cards/oracle-cards-20230414090208.json","content_type":"application/json","content_encoding":"gzip"},{"object":"bulk_data","id":"6bbcf976-6369-4401-88fc-3a9e4984c305","type":"unique_artwork","updated_at":"2023-04-14T09:02:47.020+00:00","uri":"https://api.scryfall.com/bulk-data/6bbcf976-6369-4401-88fc-3a9e4984c305","name":"Unique Artwork","description":"A JSON file of Scryfall card objects that together contain all unique artworks. The chosen cards 

In [4]:
# Checking if this format gets a response from the API. Looks good!
r.status_code

200

## Putting it all Together (AKA ~Magic~ Python: The Gathering)

This is where the magic (hah) happens. Now that we have a connection to the API, we can write these card entries to a Pandas dataframe. First, we load in our Excel spreadsheet. Then we us a "for" loop to iterate over the names in the spreadsheet.

Once we have names, we create a blank "prices" list, then generate URLs based on card name--this is accomplished using an F-string.

In the same "for" loop, we also query the API, retrieve a response in JSON, and append the 'price' column entry for that card.

Last, we output to a Pandas dataframe and check the results. Looking good!

In [43]:
# Importing all libraries again in one cell so everything can be exported to Power BI
import requests
import openpyxl
import pandas as pd

# Load Excel file. This code will need to be run again if the .xlsx file changes.
wb = openpyxl.load_workbook('A:/Documents/Data Science/MTG_DB/decks.xlsx', data_only=True)
ws = wb.active

# Create empty list to append results
card_data = []

# Iterate over rows to fetch prices
for row in ws.iter_rows(min_row=2, values_only=True):
    card_name = row[2]
    set_code = row[4]
    collector_number = row[5]
    
    # Define API URL for card
    url = f"https://api.scryfall.com/cards/{set_code}/{collector_number}"
    
    # Fetch card data
    try:
        response = requests.get(url)
        data = response.json()
        
    except:
        print(f"No data found for {card_name} ({set_code} {collector_number})")
        continue
        
    # Checking if card has price
    if 'prices' not in data:
        
        # Look for a different printing with a price
        prints_url = f"https://api.scryfall.com/cards/search?q=!\"{card_name}\""
        prints_response = requests.get(prints_url)
        prints_data = prints_response.json()
        for printing in prints_data['data']:
            if 'prices' in printing:
                data = printing
                break
                
    # If absolutely no price is found, the price is set to None            
    if 'prices' not in data or (data['prices']['usd'] is None and data['prices']['usd_foil'] is None): 
        price = None
    
    # Sets prices based on return of loop
    price = data['prices']['usd'] or data['prices']['usd_foil']
    
    # Return remaining columns desired
    cmc = data['cmc']
    colors = data.get('colors', 'Colorless')
    card_type = data['type_line']
    oracle = data.get('oracle_text', None)
    
    # Append entries to list
    card_data.append([card_name, price, card_type, colors, cmc, oracle])

# Create a pandas dataframe and append all entries
df = pd.DataFrame(card_data, columns=['Card Name', 'Price', 'Type', 'Colors', 'CMC', 'Oracle'])     

No price data found for Lat-Nam's Legacy  (all  30a)


In [44]:
df.head(10)

,Card Name,Price,Type,Colors,CMC,Oracle
0,Acidic Soil,1.65,Sorcery,[R],3.0,Acidic Soil deals damage to each player equal ...
1,Angel's Grace,2.66,Instant,[W],1.0,Split second (As long as this spell is on the ...
2,Anguished Unmaking,6.73,Instant,"[B, W]",3.0,Exile target nonland permanent. You lose 3 life.
3,Arcane Signet,0.88,Artifact,[],2.0,{T}: Add one mana of any color in your command...
4,Arcbond,0.46,Instant,[R],3.0,Choose target creature. Whenever that creature...
5,Battlefield Forge,0.74,Land,[],0.0,{T}: Add {C}.\n{T}: Add {R} or {W}. Battlefiel...
6,Beseech the Queen,3.39,Sorcery,[B],6.0,({2/B} can be paid with any two mana or with {...
7,Bojuka Bog,1.23,Land,[],0.0,Bojuka Bog enters the battlefield tapped.\nWhe...
8,Brash Taunter,1.52,Creature — Goblin,[R],5.0,Indestructible\nWhenever Brash Taunter is deal...
9,"Breena, the Demagogue",3.52,Legendary Creature — Bird Warlock,"[B, W]",3.0,Flying\nWhenever a player attacks one of your ...


In [47]:
# Checking for any stragglers...
df1 = df[df["Price"].isnull()]
df1.head()

,Card Name,Price,Type,Colors,CMC,Oracle


## Liftoff!

We have a lovely data set with minimal null values, all thanks to Scryfall's excellent record keeping. Never underestimate the power of nerds in large numbers!

Right off the bat, I see a few points where data will need to be cleaned:
* "Type" will need to be delimited along a few different characters to split each card into single types and subtypes.
* "Colors" will need to be split so that each instance of a color in the "WUBRG pie" is distinct.
* This information will need to be indexed against the spreadsheet from which the names were read in. Luckily Power BI sports some robust modeling options that will help solve this.

## Future Exploration

In future analysis, I might want to export this data frame to CSV. Let's take a look at a few basics.

In [9]:
# Checking for working directory to see where results will be exported.
import os
print(os.getcwd())

C:\Users\twiss\Documents


In [10]:
# Export the complete CSV to a directory for reference. Separate using tabs to create separate columns.
df.to_csv('scryfall.csv', sep='\t', index=False)